# Задание 
Аристова, Завьялова, Лобанов, `II.10.6`, вариант `о`.

$$
\begin{equation}
  \left\{ 
    \begin{aligned} 
      ax_1 + x_2 + x_3 / b = 1          \\
      x_1  + ax_2 + x_3 + x_4 / b = 2   \\
              x_2 + x_3 + x_4 / b = 3 \\
              \ldots\\
      x_{m - 1} + ax_m + x_{m+1} + x_{m + 2} / b = m\\
              \ldots \\
      x_{n - 1} + ax_{n} = n\\
    \end{aligned} 
  \right.
\end{equation}
$$

with $n = 100$, $a = b = 10$


# Решение



In [27]:
## результат решения, чтобы не листать в конец
## перед выполнением текущей ячейки, запустить все последующие ячейки\

analit_solve = []
iter_solve = SolveRelax (coef_matrix, right_side_vector)

if (can_use_LU):
    analit_solve = SolveLU (coef_matrix, right_side_vector)
else:
    analit_solve = SolveGauss (coef_matrix, right_side_vector)

print ("Неувязка для аналитического решения: ", GetHitch (coef_matrix, right_side_vector, analit_solve))
print ("Неувязка для итерационного решения: ",  GetHitch (coef_matrix, right_side_vector, iter_solve))
print ("Число обусловленности матрицы mu = ", GetMu (coef_matrix))
print ("lambda_max = ", GetLambdaMax (coef_matrix), 
     "\nlambda_min = ", GetLambdaMin (coef_matrix))

print ("\n\n\n")
print ("Аналитическое решение:\n", analit_solve)
print ("\nИтерационное решение:\n", iter_solve)

Неувязка для аналитического решения:  6.957936963747176e-14
Неувязка для итерационного решения:  8.689832922017065e-07
Число обусловленности матрицы mu =  1.4991943334502218
lambda_max =  11.999032564583974 
lambda_min =  8.000967435416026




Аналитическое решение:
 [0.08114061881585005, 0.16393716688650095, 0.24656644954998497, 0.329212627691554, 0.4118570992209515, 0.4945017431352375, 0.5771463696369266, 0.6597909978974621, 0.7424356259803365, 0.8250802540811565, 0.9077248821801637, 0.9903695102793542, 1.0730141383785259, 1.1556587664776996, 1.238303394576873, 1.3209480226760468, 1.4035926507752203, 1.4862372788743938, 1.5688819069735673, 1.6515265350727408, 1.7341711631719146, 1.8168157912710878, 1.8994604193702618, 1.982105047469435, 2.0647496755686086, 2.1473943036677827, 2.2300389317669556, 2.312683559866129, 2.395328187965303, 2.4779728160644763, 2.56061744416365, 2.6432620722628233, 2.725906700361997, 2.8085513284611707, 2.8911959565603444, 2.9738405846595177, 3.05648521275869

In [14]:
import numpy as np
from copy import deepcopy
np.set_printoptions(precision=5, suppress=True)

## Генерация матрицы

In [15]:
n = 100
a = 10
b = 10

In [16]:
coef_matrix = np.zeros((n, n), dtype=np.float64)

for line_num in range(n):    
    coef_idx = line_num
    
    if coef_idx - 1 >= 0:
        coef_matrix[line_num][coef_idx - 1] = 1
    
    coef_matrix[line_num][coef_idx] = a

    if coef_idx + 1 < n:
        coef_matrix[line_num][coef_idx + 1] = 1

    if coef_idx + 2 < n:
        coef_matrix[line_num][coef_idx + 2] = 1 / b

# checks that's all is ok
assert(np.all((coef_matrix[0] - ([10, 1, 0.1   ] + [0] * (n - 3))) == 0))
assert(np.all((coef_matrix[1] - ([1, 10, 1, 0.1] + [0] * (n - 4))) == 0))

assert(np.all(coef_matrix[-2] - ([0] * (n - 3) + [1, 10, 1]) == 0))
assert(np.all(coef_matrix[-1] - ([0] * (n - 2) + [1, 10   ]) == 0))

assert(coef_matrix[n // 2][n // 2 - 1] == 1.0)

right_side_vector = np.array([i + 1 for i in range(n)], dtype=np.float64)

## Функции для расчета неувязки

In [17]:
def Norm3Vec (x):
    return np.sqrt (np.sum (x * x))

def Norm3Matr (matr):
    w, v = np.linalg.eigh (matr)
    return np.max (np.abs (w))

def GetHitch (A, f, x):
    return Norm3Vec (np.matmul (A, x) - f)

def GetMu (A):
    return Norm3Matr (A) * Norm3Matr (np.linalg.inv (A))

def GetLambdaMax (matr):
    w, v = np.linalg.eigh (matr)
    return np.max (w)

def GetLambdaMin (matr):
    w, v = np.linalg.eigh (matr)
    return np.min (w)

## Методы решений

In [18]:
# Проверка применимости LU-разложения
can_use_LU = True

for i in range (n):
    if (np.linalg.det ([A_j[0:(i+1)] for A_j in coef_matrix[0:(i+1)]]) == 0):
        can_use_LU = False
        
print(f"LU is { 'im' if not can_use_LU else ''}possible")

LU is possible


In [19]:
def SolveGauss(A, f):
    A = deepcopy(A)
    f = deepcopy(f)

    for row_idx in range(num_equasions - 1):
        max_elem_ind = FindMaxElemIdx(A, row_idx)

        if (max_elem_ind != row_idx):
            A[i], A[max_elem_ind] = deepcopy(A[max_elem_ind]), deepcopy (A[i])
        
        for j in range(i + 1, lenA):
            coef = A[j][i] / A[i][i]
            A[j] -= A[i] * coef
            f[j] -= f[i] * coef

        # For some reason the solution below gives too large error :(
        # Thats doubly insulting bc I like it solutuion way more
        #
        # coefs = A[row_idx:, row_idx:]
        # right_side_vector = f[row_idx:]
        # max_element_col_idx = np.argmax(coefs[0])
        # A[[row_idx, row_idx + max_element_col_idx], :] = A[[row_idx + max_element_col_idx, row_idx], :]
        # 
        # multipliers = - coefs[ : , max_element_col_idx] / coefs[max_element_row_idx, max_element_row_idx]
        # multipliers[max_element_col_idx] = 0
        
        # coefs += np.outer(multipliers, coefs[:, max_element_col_idx])
        # right_side_vector +=  multipliers * right_side_vector[0]

    return ReverseStrokeU(A, f)


def FindMaxElemIdx(A, i_row):
    max_elem_ind = i_row
    for j in range (i_row + 1, len (A)):
        if (abs (A[j][i_row]) > abs (A[max_elem_ind][i_row])):
            max_elem_ind = j
    return max_elem_ind



In [26]:
def SolveLU (A_const, f_const):
    lenA = len (A_const)
    A = deepcopy (A_const)
    f = deepcopy (f_const)
    
    L = [[0 for j in range (lenA)] for i in range (lenA)]
    U = deepcopy (L)
    for i in range (lenA):
        L[i][i] = 1
    
    get_Uij = lambda i, j:  A[i][j] - sum ([L[i][k] * U[k][j] for k in range (i)])
    get_Lij = lambda i, j: (A[i][j] - sum ([L[i][k] * U[k][j] for k in range (j)])) / U[j][j]
    
    for i in range (lenA):
        for j in range (i):
            L[i][j] = get_Lij (i, j)
        
        for j in range (i, lenA):
            U[i][j] = get_Uij (i, j)
            
    # solve LY = f   
    y = ReverseStrokeL (L, f)
    # solve Ux = y
    return ReverseStrokeU (U, y)

In [22]:
def ReverseStrokeU (U, f):
    lenU = len (U)
    x = [0. for i in range (lenU)]
    
    for i in range (lenU - 1, -1, -1):
        x_i = 0.
        
        for j in range (i + 1, lenU):
            x_i += U[i][j] * x[j]
        x[i] = (f[i] - x_i) / U[i][i]
    
    return x

def ReverseStrokeL (L, f):
    x = ReverseStrokeU (np.flip (L, (0, 1)), np.flip (f))
    return np.flip (x)

In [23]:
def SolveRelax (A, f):
    lenA = len (A)
    w    = 1.726
    x_0  = np.linspace (0, 2, lenA) # random
    EPS  = 1e-6
    
    L = np.array ([[0. for j in range (lenA)] for i in range (lenA)])
    D = deepcopy (L)
    U = deepcopy (L)
    
    for i in range (lenA):
        for j in range (lenA):
            if (j > i):
                U[i][j] = A[i][j]
            elif (j < i):
                L[i][j] = A[i][j]
            elif (j == i):
                D[i][j] = A[i][j]
    
    for i in range (lenA):
        for j in range (lenA):
            test = np.abs (U[i][j] + L[i][j] + D[i][j] - A[i][j])
            if (test != 0):
                print (i, j, U[i][j], L[i][j], D[i][j], A[i][j])
                
    D_wL_pow_m1 = np.linalg.inv (D + w * L)
    wm1_D_wU    = (w - 1) * D + w * U
    
    coef_before_x = -np.matmul (D_wL_pow_m1, wm1_D_wU)
    coef_free     = w * np.matmul (D_wL_pow_m1, f)
    
    do_iter = lambda x: np.matmul (coef_before_x, x) + coef_free

    x = x_0
    while (np.abs (GetHitch (A, f, x)) > EPS):
        x = do_iter (x)
        
    return x